In [1]:
import csv

In [2]:
file = "data/skos/cv_burial.csv"

In [3]:
class Csv2SkosReader(object):
    """
    extract SKOS-like objects from special structured CSV sheets
    """
            
    def __init__(self, csv_file):
        self.csv_file = csv_file
        try:
            with open(self.csv_file, encoding='utf-8') as csvfile:
                self.data = [x for x in csv.reader(csvfile, delimiter=',')]
        except:
            print('could not open csv file')
        self.headers = self.data[0]
        try:
            self.alt_lang = (self.headers[1])[(self.headers[1]).index('@')+1:]
        except:
            self.alt_lang = None
        self.schemes = set([x[0] for x in self.data[1:]])
        self.number_of_schemes = len(self.schemes)
        
    def get_concepts(self):
        concepts = [] 
        for x in self.data[1:]:
            first_order = x[1].split('|')
            if x[2] != '':
                second_order = x[2].split('|')
                concept = {
                    'scheme': x[0],
                    'concept': {
                        'pref_label': first_order[0],
                        'pref_label_lang': 'eng',
                        'alt_label': self.alt_lang,
                        'alt_label_lang': self.alt_lang,
                        'narrower':{
                            'scheme': x[0],
                            'concept': {
                                'pref_label': second_order[0],
                                'pref_label_lang': 'eng',
                                'alt_label': second_order[1],
                                'alt_label_lang': self.alt_lang,
                            }
                        }
                    }
                }
            else:
                concept = {
                    'scheme': x[0],
                    'concept': {
                        'pref_label': first_order[0],
                        'pref_label_lang': 'eng',
                        'alt_label': first_order[1],
                        'alt_label_lang': self.alt_lang,
                    }
                }
            concepts.append(concept)
            
        return concepts         

In [4]:
class Csv2SkosImporter(Csv2SkosReader):
    """Takes a special formatted csv file, parses it and imports the derived data into vocabs"""
    
    def update_schemes(self):
        """import/updates all conceptSchemes found in csv"""
        report = {}
        report['before'] = len(SkosConceptScheme.objects.all())
        failed = []
        success = []
        for x in self.schemes:
            try:
                clean = x.split('|')[0].strip()
            except:
                clean = x.strip()
            try:
                temp_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title=clean)
                temp_scheme.save()
                success.append(x)
            except:
                failed.append(x)
        report['failed'] = failed
        report['success'] = success
        report['after'] = len(SkosConceptScheme.objects.all())
        return report
    
    def update_concepts(self):
        """import/updates all SkosConcepts found in csv"""
        report = {}
        report['before'] = len(SkosConcept.objects.all())
        report['schemes_before'] = len(SkosConceptScheme.objects.all())
        failed = []
        success = []
        for x in self.get_concepts():
            #get scheme
            try:
                clean = x['scheme'].split('|')[0].strip()
            except:
                clean = x['scheme'].strip()
            temp_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title=clean)
            # crete 1st order
            try:
                temp_label, _ = SkosLabel.objects.get_or_create(
                    label=x['concept']['alt_label'],
                    label_type='altLabel',
                    isoCode=x['concept']['alt_label_lang']
                )
                temp_first, _ = SkosConcept.objects.get_or_create(
                    pref_label=x['concept']['pref_label'],
                    pref_label_lang=x['concept']['pref_label_lang']
                )
                temp_first.label=[temp_label]
                temp_first.scheme = [temp_scheme]
                success.append(x['concept']['pref_label'])
            except:
                failed.append(x['concept']['pref_label'])
            try:
                second = x['concept']['narrower']['concept']
                    # crete 1st order
                try:
                    temp_label, _ = SkosLabel.objects.get_or_create(
                        label=second['alt_label'],
                        label_type='altLabel',
                        isoCode=second['alt_label_lang']
                    )
                    temp_second, _ = SkosConcept.objects.get_or_create(
                        pref_label=second['pref_label'],
                        pref_label_lang=second['pref_label_lang']
                    )
                    temp_second.label=[temp_label]
                    temp_second.scheme = [temp_scheme]
                    temp_first.skos_narrower = [temp_second]
                    success.append(second['pref_label'])
                except:
                    failed.append(second['pref_label'])
            except:
                pass
        report['failed'] = failed
        report['success'] = success
        report['after'] = len(SkosConcept.objects.all())
        report['schemes_after'] = len(SkosConceptScheme.objects.all())
        return report

In [5]:
hansi = Csv2SkosImporter('data/skos/cv_burial.csv')

In [79]:
hansi.update_concepts()

{'after': 211,
 'before': 211,
 'failed': [],
 'schemes_after': 24,
 'schemes_before': 24,
 'success': ['hillside',
  'on a terrace',
  'at the coast',
  'immediate vincinity to a river',
  'in a valley',
  'on a hill crest',
  'open plain',
  'no corresponding settlement',
  '<100 m',
  '<500 m',
  '<1000 m',
  '<3000 m',
  'tumuli cemetery',
  'burial site with tumuli and shallow graves',
  'flat grave cemetery',
  'Bz A',
  'Bz B',
  'Bz C',
  'BZ D',
  'Ha A1',
  'Ha A2',
  'Ha B1',
  'Ha B2',
  'Ha B3',
  'Ha C1',
  'burial mound',
  'enclosure',
  'rock chamber',
  'soil',
  'stone',
  'soil/stone',
  'wood',
  'cremation burial',
  'no choice',
  'cremation burial',
  'urn burial',
  'cremation burial',
  'urn burial',
  'cremation burial',
  'urn burial',
  'cremation burial',
  'urn burial',
  'cremation burial',
  'cremation burial without an urn',
  'cremation burial',
  'cremation burial without an urn',
  'cremation burial',
  'cremation burial without an urn',
  'crematio

In [11]:
hansi.get_concepts()

[{'concept': {'alt_label': 'am Hang',
   'alt_label_lang': 'ger',
   'pref_label': 'hillside',
   'pref_label_lang': 'eng'},
  'scheme': 'Topography|Topographie'},
 {'concept': {'alt_label': 'an der Hangterrasse',
   'alt_label_lang': 'ger',
   'pref_label': 'on a terrace',
   'pref_label_lang': 'eng'},
  'scheme': 'Topography|Topographie'},
 {'concept': {'alt_label': 'an der Küste',
   'alt_label_lang': 'ger',
   'pref_label': 'at the coast',
   'pref_label_lang': 'eng'},
  'scheme': 'Topography|Topographie'},
 {'concept': {'alt_label': 'unmittelbare Flußnähe/Fließgewässer',
   'alt_label_lang': 'ger',
   'pref_label': 'immediate vincinity to a river',
   'pref_label_lang': 'eng'},
  'scheme': 'Topography|Topographie'},
 {'concept': {'alt_label': 'im Tal',
   'alt_label_lang': 'ger',
   'pref_label': 'in a valley',
   'pref_label_lang': 'eng'},
  'scheme': 'Topography|Topographie'},
 {'concept': {'alt_label': 'Hügelkuppe',
   'alt_label_lang': 'ger',
   'pref_label': 'on a hill crest'

In [40]:
for x in SkosConceptScheme.objects.all():
    x.delete()
    #print(x)

In [25]:
print(SkosConceptScheme.objects.all())

[]


In [81]:
for x in SkosConcept.objects.all():
    x.delete()